# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
import ipywidgets as widgets

In [2]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment

# Get the openai key

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key and openai_api_key.startswith('sk-proj-') and len(openai_api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()
# Get the ollama key using the llama model

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')



API key looks good so far


In [5]:
system_prompt = "You are a knowledgeable technical instructor who helps students understand \
complex concepts across a wide range of technical topics. Your expertise includes artificial]\
intelligence, machine learning, large language models (LLMs), and programming in languages \
such as Python, JavaScript, Java, and more. You also provide in-depth support for \
AI engineering questions and other advanced technical subjects."

In [6]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
# Helper to answer using selected model without duplicating logic

def answer_question(question_text: str, model_choice: str):
    if not question_text or not question_text.strip():
        display(Markdown("**Please enter a question.**"))
        return

    if model_choice == MODEL_GPT:
        stream = openai.chat.completions.create(
            model=MODEL_GPT,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question_text},
            ],
            stream=True,
        )
        response_text = ""
        handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response_text += chunk.choices[0].delta.content or ""
            response_text = response_text.replace("```", "").replace("markdown", "")
            update_display(Markdown(response_text), display_id=handle.display_id)
        return

    if model_choice == MODEL_LLAMA:
        response = ollama_via_openai.chat.completions.create(
            model=MODEL_LLAMA,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question_text},
            ],
        )
        result = response.choices[0].message.content
        display(Markdown(result))
        return

    display(Markdown(f"Unknown model: {model_choice}"))


In [8]:
# Get gpt-4o-mini to answer, with streaming
answer_question(question, MODEL_GPT)

The code you provided is a Python statement that uses the `yield from` expression in conjunction with a set comprehension. Let’s break it down step-by-step to understand what it does and why it’s written that way.

### Breakdown of the Code

1. **Set Comprehension**:
   python
   {book.get("author") for book in books if book.get("author")}
   
   - This part of the code is a set comprehension. It constructs a set containing authors' names.
   - `books` is expected to be an iterable (like a list) of dictionaries, where each dictionary represents a book.
   - `book.get("author")` attempts to retrieve the value associated with the key "author" from each `book` dictionary.
   - The `if book.get("author")` condition filters out any books where the "author" key does not exist or has a value that evaluates to `False` (like `None` or an empty string). Only books with valid authors are included in the resulting set.

2. **Yielding**:
   python
   yield from ...
   
   - The `yield from` expression is used within a generator function. It allows the generator to yield all values from another iterable (in this case, the set created by our set comprehension).
   - When this statement is executed in the context of a generator function, it will yield all unique author names from the `books` collection.

### Why Use This Code?

1. **Efficiency**: Using a set comprehension guarantees that only unique authors are collected since sets do not allow duplicate elements. If you only wanted distinct authors, this is a concise and efficient way to do so.

2. **Clarity**: By using `yield from`, the code clearly communicates that the generator will produce values from whatever is being yielded, in this case, the unique author names.

3. **Generator Functionality**: This approach allows the code to be part of a larger generator function, which can provide the authors one-by-one on-demand (lazy evaluation), rather than all at once. This can be particularly useful if you're processing a large number of books and want to optimize memory usage.

### Example Usage

Here’s an example of how this might look in a full generator function:

python
def authors_generator(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Usage example
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},
    {"title": "Book 4"}  # No author
]

for author in authors_generator(books):
    print(author)


### Output

This would output:

Author A
Author B


In summary, the code collects and yields unique author names from a collection of book dictionaries, helping you efficiently manage the data you work with.

In [9]:
# Get Llama 3.2 to answer
answer_question(question, MODEL_LLAMA)

**Decompiling the Code**

This line of Python code uses a concept called "context managers" and "generators." Let's break it down:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's what each part does:

* `books` is an iterable (e.g., list, tuple, dictionary, set) containing objects with attributes or "key-value pairs."
* `{}` - This represents a dictionary comprehension.
	+ `.get()` is a method extracted from the dictionaries in the `books` iterable. It retrieves the value associated with the string key `"author"` from each dictionary, allowing it to return `None` if no such key exists (`book.get("author")`)
* `for book in books`: This loop iterates over each object (dictionary) in the `books` iterable.
	+ Inside this loop, `.get("author")` is called for each dictionary, just like above. This extracts the author name as a string.
* `yield from {...}`: This expression uses the `yield from` syntax, which introduces a "generator" - an object that defines how to generate a sequence of values.

**How it works**

1. The code creates a context manager for each dictionary in `books`.
2. For each dictionary using `.get("author")`, it captures the resulting value and stores it inside the `{...}` dictionary comprehension.
3. When this expression is executed, it yields all the captured values (the author names) as if it were yielding from an iterator.

**Equivalent Code**

A more explicit equivalent of the code above would be:

```python
author_names = []
for book in books:
    author_name = book.get("author")
    if author_name:  # only if a key exists
        author_names.append(author_name)
for name in author_names:
    yield name
```

However, this example does not utilize Python generators or comprehension, which is the purpose of the original line.

**Use Cases and Performance Benefits**

This kind of expression can be beneficial for several use cases:

*   Generating large amounts of data: If you have a large dataset, a generator expression like `yield from {...}` may conserve more memory than a traditional list comprehension. The latter stores its entire output in RAM before using it.
*   Handling infinite sequences: When dealing with potentially endless iterators (like those produced by certain types of algorithms), this syntax allows you to implement iterative processing without running out of memory.
*   Creating iterators for data structures: This syntax helps reduce the necessity of manual iterator handling when working with collections or lists.

In general, `yield from` expressions are often employed when tasks imply large datasets or sequential processing. They help reduce memory costs and improve overall code performance.

In [10]:
# ipywidgets UI

model_dropdown = widgets.Dropdown(
    options=[("GPT", MODEL_GPT), ("LLaMA", MODEL_LLAMA)],
    value=MODEL_GPT,
    description="Model:",
)

question_area = widgets.Textarea(
    value="",
    placeholder="Ask your technical question here...",
    description="Question:",
    layout=widgets.Layout(width='100%', height='120px')
)

ask_button = widgets.Button(description="Ask", button_style="primary")
output = widgets.Output()


def on_ask_clicked(_):
    output.clear_output()
    with output:
        answer_question(question_area.value, model_dropdown.value)


ask_button.on_click(on_ask_clicked)

ui = widgets.VBox([
    model_dropdown,
    question_area,
    ask_button,
    output,
])

ui
